In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
data = pd.read_csv('music_data.csv')

In [24]:
data.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date
0,Beete Lamhein,KK,The Train,1GFvChouNq6BNSjhSi6XKP,0zQa7QXLpUZfrrsWbgDZll,64,2007-04-18
1,Ik Bagal,Piyush Mishra,Gangs Of Wasseypur,5lOKYvuSy6buolqIHSUwpG,4OOXIr4Wce7TuLUu65v653,48,2012-05-23
2,"Kya Mujhe Pyar Hai (From ""Woh Lamhe"")",KK,Evergreen Hits Of K.K.,1dG5jJAa3ZiSWnpVd2uPao,7fW9J3EpVWVN1ouv0slAs0,65,2022-06-01
3,Dhoom Taana,"Vishal-Shekhar, Shreya Ghoshal, Abhijeet",Om Shanti Om,2cUzlmLfL5LUTSEk7qG09k,4eKWtXoGajnaGKvKAcgc8f,57,2007-09-18
4,Chahun Main Ya Naa,"Palak Muchhal, Arijit Singh",Aashiqui 2,0Rkv5iqjF2uenfL0OVB8hg,6UgcN95w7vQxkR8sEFmwHG,68,2013-04-06


In [25]:
data['Info'] = data['Track Name'] + " " + data['Artists'] + " " + data['Album Name']

In [26]:
data.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Info
0,Beete Lamhein,KK,The Train,1GFvChouNq6BNSjhSi6XKP,0zQa7QXLpUZfrrsWbgDZll,64,2007-04-18,Beete Lamhein KK The Train
1,Ik Bagal,Piyush Mishra,Gangs Of Wasseypur,5lOKYvuSy6buolqIHSUwpG,4OOXIr4Wce7TuLUu65v653,48,2012-05-23,Ik Bagal Piyush Mishra Gangs Of Wasseypur
2,"Kya Mujhe Pyar Hai (From ""Woh Lamhe"")",KK,Evergreen Hits Of K.K.,1dG5jJAa3ZiSWnpVd2uPao,7fW9J3EpVWVN1ouv0slAs0,65,2022-06-01,"Kya Mujhe Pyar Hai (From ""Woh Lamhe"") KK Everg..."
3,Dhoom Taana,"Vishal-Shekhar, Shreya Ghoshal, Abhijeet",Om Shanti Om,2cUzlmLfL5LUTSEk7qG09k,4eKWtXoGajnaGKvKAcgc8f,57,2007-09-18,"Dhoom Taana Vishal-Shekhar, Shreya Ghoshal, Ab..."
4,Chahun Main Ya Naa,"Palak Muchhal, Arijit Singh",Aashiqui 2,0Rkv5iqjF2uenfL0OVB8hg,6UgcN95w7vQxkR8sEFmwHG,68,2013-04-06,"Chahun Main Ya Naa Palak Muchhal, Arijit Singh..."


In [27]:
vector = TfidfVectorizer(stop_words='english')

In [28]:
tfid_matrix = vector.fit_transform(data['Info'])

In [29]:
cosin = cosine_similarity(tfid_matrix,tfid_matrix)

,Track Name,Artist,Album Name,Release Date,Cosine Similarity
0,Tum Ho Mera Pyar,"KK, Suzanne",Haunted,2011-04-13,0.213700
1,Beete Lamhein,KK,The Train,2007-04-18,0.113687
2,Dil Ibaadat,"Pritam, KK",Tum Mile (Original Motion Picture Soundtrack),2009-09-08,0.101990
3,Hona Tha Pyar,"Atif Aslam, Hadiqa Kiani",Bol (Original Motion Picture Soundtrack),2011-06-05,0.101842
4,Naina Da Kya Kasoor,Amit Trivedi,Andhadhun (Original Motion Picture Soundtrack),2018-10-03,0.100731


In [38]:
def recommend_tracks_by_song_name(song_name, cosin, top_n=3):
    track_index = data[data['Track Name'].str.lower() == song_name.lower()]
    
    if track_index.empty:
        return "Song not found in the dataset."
    track_index = track_index.index[0]
    sim_scores = list(enumerate(cosin[track_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  
    recommended_tracks = []
    for i, score in sim_scores:
        track_info = data.iloc[i]
        recommended_tracks.append({
            'Track Name': track_info['Track Name'],
            'Artist': track_info['Artists'],
            'Album Name': track_info['Album Name'],
            'Release Date': track_info['Release Date'],
            'Cosine Similarity': score
        })
    recommended_df = pd.DataFrame(recommended_tracks)
    
    return recommended_df

In [39]:
song_name = input("Enter the song name to get recommendations: ")
recommended_tracks = recommend_tracks_by_song_name(song_name, cosin)

if isinstance(recommended_tracks, str):  
    print(recommended_tracks)
else:
    print(recommended_tracks)

Enter the song name to get recommendations:  Dil Ibaadat


         Track Name                                Artist  \
0  Tum Hi Ho Bandhu  Neeraj Shridhar, Kavita Seth, Pritam   
1  Tum Ho Mera Pyar                           KK, Suzanne   
2       Be Intehaan   Atif Aslam, Sunidhi Chauhan, Pritam   

                                      Album Name Release Date  \
0  Cocktail (Original Motion Picture Soundtrack)   2012-06-15   
1                                        Haunted   2011-04-13   
2    Race 2 (Original Motion Picture Soundtrack)   2012-01-25   

   Cosine Similarity  
0           0.248155  
1           0.235625  
2           0.221800  
